In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:

import sys
sys.path.append('../../')
import random
import warnings

from src.utils import read_json_df
from src.models.median_baseline import MedianBaselineModel

from src.models.factorization import FunkSVDModel, AlternatingLeastSquaresModel
from src.models.nn import NNColaborativeModel
import pandas as pd
#from bandit import GreadyBandit,ThompsonBandit
warnings.filterwarnings("ignore")

In [18]:
review_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_review.json")
user_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_user.json")
business_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_business.json")

In [19]:
review_df = review_df[review_df.date < ' -01-01']


In [20]:
review_df = review_df.sort_values('date')

In [21]:
train_df,test_df = review_df[review_df.date < '2014-01-01'],review_df[review_df.date >= '2014-01-01']

In [22]:
user_df = user_df[user_df.user_id.isin(review_df.user_id.unique())]
business_df = business_df[business_df.business_id.isin(review_df.business_id.unique())]


In [23]:
user_df = user_df.reset_index(drop=True).reset_index()
business_df = business_df.reset_index(drop=True).reset_index()


In [24]:
from src.models.sequential_model import SequentialModel
model_seq = SequentialModel()


In [25]:
model_seq.evaluate(review_df, user_df, business_df)

Evaluation fold: 100%|███████████████████████████| 5/5 [36:57<00:00, 443.59s/it]


{'rmse': np.float64(1.3844743039200635),
 'mae': np.float64(1.033851573015794),
 'accuracy': 0.29164115206290575,
 'f1': np.float64(0.09016305401990808),
 'precision': np.float64(0.8583282304125811),
 'recall': np.float64(0.2),
 'AP@1': np.float64(0.0009432805490936615),
 'AP@3': np.float64(0.000923088055056138),
 'AP@K': np.float64(0.000897166025676196),
 'MAP@K': np.float64(0.0009371031867351921)}

In [26]:
median = MedianBaselineModel()

In [27]:
median.evaluate(review_df, user_df, business_df)

Evaluation fold: 100%|████████████████████████████| 5/5 [01:23<00:00, 16.74s/it]


{'rmse': np.float64(1.3610127825154232),
 'mae': np.float64(0.9589928552555484),
 'accuracy': 0.3645783282621986,
 'f1': np.float64(0.26969697466078174),
 'precision': np.float64(0.3587912378303729),
 'recall': np.float64(0.27759942335166415),
 'AP@1': np.float64(4.501981642795329e-05),
 'AP@3': np.float64(4.431728741866215e-05),
 'AP@K': np.float64(3.842530676517355e-05),
 'MAP@K': np.float64(4.137579798205162e-05)}

As we can see, the Sequence model gives results much higher than the Median Model data, and the results of this model are also higher than the PageRank and Factorization models. 
For me, these results are expected because the Sequential Model is designed specifically for such tasks.
This type of model tries to predict the next value based on the previous ones, and also takes into account not only the visited restaurants, but also the order in which they were visited.